In [ ]:
import nltk

# 8.1 一些语法困境

## 8.1.1 普遍存在的歧义

In [3]:
groucho_grammar = nltk.CFG.fromstring('''
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | "I"
VP -> V NP | VP PP
Det -> "an" | "my"
N -> "elephant" | "pajamas"
V -> "shot"
P -> "in"
''')

In [5]:
# 基于一种文法解析句子，可能会解析出两种结构
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)  # 图解析
for i, tree in enumerate(parser.parse(sent)):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


# 8.3 上下文无关文法（context-free grammars，CFG）

## 8.3.1 一种简单的文法

In [6]:
# Ex8-1 一个简单的上下文无关文法的例子
grammar1 = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "a" | "an" | "the" | "my" | "The"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"
""")

In [7]:
# 句子剖析会出现两个符合文法规则的结果，称为结构上有歧义。这个歧义称为介词短语附着歧义。
sent = 'The dog saw a man in the park'.split()
rd_parser = nltk.RecursiveDescentParser(grammar1)  # 递归下降解析器 RecursiveDescentParser()
for i, tree in enumerate(rd_parser.parse(sent)):
    print(tree)

(S
  (NP (Det The) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man) (PP (P in) (NP (Det the) (N park))))))
(S
  (NP (Det The) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man))
    (PP (P in) (NP (Det the) (N park)))))


In [8]:
nltk.app.rdparser()  # 通过这个演示可以辅助理解从顶向下的回溯策略的句法剖析过程

### 8.3.2 编写自己的文法

In [11]:
# 在文本文件创建和编辑语法会更加文法，然后可以利用函数加载到NLTK中进行解析
grammar1 = nltk.data.load('mygrammar.cfg')
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)  # trace = 2 不知道有何作用
for i, tree in enumerate(rd_parser.parse(sent)):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [12]:
# 输出文法文件中的内容
for p in grammar1.productions():
    print(p)

S -> NP VP
VP -> V NP
VP -> V NP PP
PP -> P NP
V -> 'saw'
V -> 'ate'
V -> 'walked'
NP -> 'John'
NP -> 'Mary'
NP -> 'Bob'
NP -> Det N
NP -> Det N PP
Det -> 'a'
Det -> 'an'
Det -> 'the'
Det -> 'my'
Det -> 'The'
N -> 'man'
N -> 'dog'
N -> 'cat'
N -> 'telescope'
N -> 'park'
P -> 'in'
P -> 'on'
P -> 'by'
P -> 'with'


### 8.3.3 句法结构中的递归

RecursiveDescentParser()无法处理形如X→XY的左递归产生式

In [13]:
# Ex-2：递归的上下文无关文法
grammar2 = nltk.CFG.fromstring("""
S  -> NP VP
NP -> Det Nom | PropN
Nom -> Adj Nom | N
VP -> V Adj | V NP | V S | V NP PP
PP -> P NP
PropN -> 'Buster' | 'Chatterer' | 'Joe'
Det -> 'the' | 'a'
N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
Adj  -> 'angry' | 'frightened' |  'little' | 'tall'
V ->  'chased'  | 'saw' | 'said' | 'thought' | 'was' | 'put'
P -> 'on'
""")

In [14]:
sent = 'the angry bear chased the frightened little squirrel'.split()
# RecursiveDescentParser()无法处理形如X→XY的左递归产生式
rd_parser = nltk.RecursiveDescentParser(grammar2)
for i, tree in enumerate(rd_parser.parse(sent)):
    print(tree)

(S
  (NP (Det the) (Nom (Adj angry) (Nom (N bear))))
  (VP
    (V chased)
    (NP
      (Det the)
      (Nom (Adj frightened) (Nom (Adj little) (Nom (N squirrel)))))))


In [ ]:
sent = 'Chatterer said Buster thought the tree was tall'.split()
rd_parser = nltk.RecursiveDescentParser(grammar2)
for i, tree in enumerate(rd_parser.parse(sent)):

    print(tree)

--------------- >第 1 个结构< ---------------
(S
  (NP (PropN Chatterer))
  (VP
    (V said)
    (S
      (NP (PropN Buster))
      (VP
        (V thought)
        (S (NP (Det the) (Nom (N tree))) (VP (V was) (Adj tall)))))))


## 8.4 上下文无关文法分析

In [15]:
grammar1 = nltk.CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> "saw" | "ate" | "walked"
NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
Det -> "a" | "an" | "the" | "my" | "The"
N -> "man" | "dog" | "cat" | "telescope" | "park"
P -> "in" | "on" | "by" | "with"
""")

In [16]:
sent = 'Mary saw a dog'.split()
sr_parser = nltk.ShiftReduceParser(grammar1)
for i, tree in enumerate(sr_parser.parse(sent)):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [17]:
# 跟踪解析的过程
sent = 'Mary saw a dog'.split()
sr_parser = nltk.ShiftReduceParser(grammar1,trace = 2)
for i, tree in enumerate(sr_parser.parse(sent)):
    print(tree)

Parsing 'Mary saw a dog'
    [ * Mary saw a dog]
  S [ 'Mary' * saw a dog]
  R [ NP * saw a dog]
  S [ NP 'saw' * a dog]
  R [ NP V * a dog]
  S [ NP V 'a' * dog]
  R [ NP V Det * dog]
  S [ NP V Det 'dog' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [18]:
nltk.app.srparser()  # 通过这个演示可以辅助理解自底向上的称进归约的句法剖析过程

### 8.4.4 符合语句规则的子串表（WFST）

上述简单的解析器都存在完整性和效率问题，下面将基于图表分析：即利用动态规划算法来解决这些问题

动态规划算法存储中间结果，并且在适当的时候重用，从而显著提高了效率。

WFST的缺点：
-   WFST本身不是一个分析树
-   每个非词汇文法生产式都必须是二元的
-   作为一个自下而上的文法，潜在地存在着浪费，因为它会在不符合文法的地方提出成分，后面又会放弃掉错误的成分
-   WFST并不能表示句子中的结构歧义（如两个动词短语的读取）


In [20]:
# 可以在文法中直接查找文本中单词所属类别
# lhs : left-hand-side; rhs : right-hand-side
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
productions = groucho_grammar.productions(rhs=text[3])
for product in productions:
    print(product)
    print(product.lhs())
    print(product.rhs())

N -> 'elephant'
N
('elephant',)


In [21]:
# Ex8-3 使用符合语句规则的子串表接收器
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens + 1)] for j in range(numtokens + 1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i + 1] = productions[0].lhs()
    return wfst

In [22]:
def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens + 1):
        for start in range(numtokens + 1 - span):
            end = start + span
            for mid in range(start + 1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1, nt2) in index:
                    wfst[start][end] = index[(nt1, nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" %
                              (start, nt1, mid, nt2, end, start, index[(nt1, nt2)], end))
    return wfst

In [23]:
def display(wfst, tokens):
    print('\nWFST ' + ' '.join([("%-4d" % i) for i in range(1, len(wfst))]))
    for i in range(len(wfst) - 1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or '.'), end=" ")
        print()

In [25]:
tokens = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)

[[None, NP, None, None, None, None, None, None], [None, None, V, None, None, None, None, None], [None, None, None, Det, None, None, None, None], [None, None, None, None, N, None, None, None], [None, None, None, None, None, P, None, None], [None, None, None, None, None, None, Det, None], [None, None, None, None, None, None, None, N], [None, None, None, None, None, None, None, None]]

WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    


In [26]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    S    
1    .    V    .    VP   .    .    VP   
2    .    .    Det  NP   .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


In [27]:
# 显示剖析过程
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)
display(wfst1, tokens)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]

WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    S    
1    .    V    .    VP   .    .    VP   
2    .    .    Det  NP   .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


In [29]:
nltk.app.chartparser()

TypeError: app() takes 0 positional arguments but 2 were given

# 8.5 依存关系 和 依存文法
短语结构文法：描述句子中的词和词序列的结合方式

-   依存文法：描述词与其他词之间的关系
    -    依存关系是一个中心词与其从属之间二元非对称关系。
        -   一个句子的中心词通常是动词，其他词直接依赖于中心词或者通过某些路径依赖于中心词

In [19]:
# 下面是NLTK为依存文法编码的一种方式，只能捕捉依存关系信息，不能指定依存关系的类型
groucho_dep_grammar = nltk.DependencyGrammar.fromstring("""
'shot' -> 'I' | 'elephant' | 'in'
'elephant' -> 'an' | 'in'
'in' -> 'pajamas'
'pajamas' -> 'my'
""")

In [ ]:
# 依存关系图是一个投影，若所有的词都按照线性顺序书写，则用边连接这些词并且保证所有的边不交叉。
# 一个词及其所有子节点在句子中形成了一个连续的词序列。
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)
sent = 'I shot an elephant in my pajamas'.split()
tree = []
for i, tree in enumerate(pdp.parse(sent)):
    print(tree)
tree.draw()

(shot I (elephant an (in (pajamas my))))
(shot I (elephant an) (in (pajamas my)))


# 8.6 文法开发
解析器根据短语结构文法在句子上建立树。

## 8.6.1 树库 和 文法：使用宾州树库

In [39]:
from nltk.corpus import treebank

t = treebank.parsed_sents('wsj_0001.mrg')[0]
print(t)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


In [40]:
# Ex 8-4 搜索树库找出句子的补语
def filter(tree):
    child_nodes = [child.label() for child in tree if isinstance(child, nltk.Tree)]
    return (tree.label() == 'VP') and ('S' in child_nodes)

In [43]:
VPS = [
    subtree 
    for tree in treebank.parsed_sents() 
    for subtree in tree.subtrees(filter)]
print(VPS[0])
VPS[1].draw()

(VP
  (VBN named)
  (S
    (NP-SBJ (-NONE- *-1))
    (NP-PRD
      (NP (DT a) (JJ nonexecutive) (NN director))
      (PP
        (IN of)
        (NP (DT this) (JJ British) (JJ industrial) (NN conglomerate))))))


In [44]:
# Prepositional Phrase Attachment Corpus. 介词短语附着语料库，是特别动词配价的信息源
# 搜索语料库，找出具有固定介词和名词和介词短语对，其中介绍短语附着到VP还是NP由选择的动词决定
from collections import defaultdict

entries = nltk.corpus.ppattach.attachments('training')
table = defaultdict(lambda: defaultdict(set))
for entry in entries:
    key = entry.noun1 + '-' + entry.prep + '-' + entry.noun2
    table[key][entry.attachment].add(entry.verb)

for key in sorted(table):
    if len(table[key]) > 1:
        print(key, 'N: ', sorted(table[key]['N']), 'V:', sorted(table[key]['V']))

%-below-level N:  ['left'] V: ['be']
%-from-year N:  ['was'] V: ['declined', 'dropped', 'fell', 'grew', 'increased', 'plunged', 'rose', 'was']
%-in-August N:  ['was'] V: ['climbed', 'fell', 'leaping', 'rising', 'rose']
%-in-September N:  ['increased'] V: ['climbed', 'declined', 'dropped', 'edged', 'fell', 'grew', 'plunged', 'rose', 'slipped']
%-in-week N:  ['declined'] V: ['was']
%-to-% N:  ['add', 'added', 'backed', 'be', 'cut', 'go', 'grow', 'increased', 'increasing', 'is', 'offer', 'plummet', 'reduce', 'rejected', 'rise', 'risen', 'shaved', 'wants', 'yield', 'zapping'] V: ['fell', 'rise', 'slipped']
%-to-million N:  ['declining'] V: ['advanced', 'climbed', 'cutting', 'declined', 'declining', 'dived', 'dropped', 'edged', 'fell', 'gained', 'grew', 'increased', 'jump', 'jumped', 'plunged', 'rising', 'rose', 'slid', 'slipped', 'soared', 'tumbled']
1-to-21 N:  ['dropped'] V: ['dropped']
1-to-33 N:  ['gained'] V: ['dropped', 'fell', 'jumped']
1-to-4 N:  ['added'] V: ['gained']
1-to-47 N: 

In [50]:
print("key=", key)
print("table[key]= ", table[key])
print("table['zip-in-way']= ", table['zip-in-way'])
print("len(table[key])= ", len(table[key]))
print("table['access-to-AZT']= ", table['access-to-AZT'])
print("table['offer-from-group']= ", table['offer-from-group'])

key= zip-in-way
table[key]=  defaultdict(<class 'set'>, {'V': {'requires'}})
table['zip-in-way']=  defaultdict(<class 'set'>, {'V': {'requires'}})
len(table[key])=  1
table['access-to-AZT']=  defaultdict(<class 'set'>, {'V': {'had'}, 'N': {'had'}})
table['offer-from-group']=  defaultdict(<class 'set'>, {'V': {'received'}, 'N': {'rejected'}})


In [51]:
# 现代汉语中央研究院平衡语料库中的10000句已经分析的句子
nltk.corpus.sinica_treebank.parsed_sents()[3450].draw()

## 8.6.2 有害的歧义

In [52]:
grammar = nltk.CFG.fromstring("""
S -> NP V NP
NP -> NP Sbar
Sbar -> NP V
NP -> 'fish'
V -> 'fish'
""")

In [ ]:
# 当fish的数量为（3，5，7...），分析树的数量是（1，2，5...），这是Catalan数
tokens = ['fish'] * 5
cp = nltk.ChartParser(grammar)
for i, tree in enumerate(cp.parse(tokens)):

    print(tree)
tree.draw()

--------------- >第 1 个结构< ---------------
(S (NP fish) (V fish) (NP (NP fish) (Sbar (NP fish) (V fish))))
--------------- >第 2 个结构< ---------------
(S (NP (NP fish) (Sbar (NP fish) (V fish))) (V fish) (NP fish))


## 8.6.3 加权语法

In [55]:
# Ex8-5: 宾州树库样本中 give 和 gave 的用法
# 检查所有包含 give 介词格和双宾语结构的实例
def give(t):
    result = t.label() == 'VP' and len(t) > 2 and t[1].label() == 'NP'  # give 双宾语结构
    result = result and (t[2].label() == 'PP-DTV' or t[2].label() == 'NP')  # give 介绍格
    result = result and ('give' in t[0].leaves() or 'gave' in t[0].leaves())  # give和gave的用法
    return result

In [56]:
def sent(tree):
    return ' '.join(token for token in tree.leaves() if token[0] not in '*-0')

In [57]:
def print_node(t, width):
    output = '%s %s: %s / %s: %s' % (
        sent(t[0]), t[1].label(), sent(t[1]), t[2].label(), sent(t[2]))
    if len(output) > width:
        output = output[:width] + '...'
    print(output)

In [58]:
for tree in nltk.corpus.treebank.parsed_sents():
    for t in tree.subtrees(give):
        print_node(t, 72)
print(t)

gave NP: the chefs / NP: a standing ovation
give NP: advertisers / NP: discounts for maintaining or increasing ad sp...
give NP: it / PP-DTV: to the politicians
gave NP: them / NP: similar help
give NP: them / NP: 
give NP: only French history questions / PP-DTV: to students in a Europe...
give NP: federal judges / NP: a raise
give NP: consumers / NP: the straight scoop on the U.S. waste crisis
gave NP: Mitsui / NP: access to a high-tech medical product
give NP: Mitsubishi / NP: a window on the U.S. glass industry
give NP: much thought / PP-DTV: to the rates she was receiving , nor to ...
give NP: your Foster Savings Institution / NP: the gift of hope and free...
give NP: market operators / NP: the authority to suspend trading in futu...
gave NP: quick approval / PP-DTV: to $ 3.18 billion in supplemental appr...
give NP: the Transportation Department / NP: up to 50 days to review any...
give NP: the president / NP: such power
give NP: me / NP: the heebie-jeebies
give NP: holders / NP: 

In [59]:
# Ex8-6: 定义一个概率上下文无关文法（PCFG）
# 只是演示了一个概率上下文无关文法的作用，也是个玩具文法
grammar = nltk.PCFG.fromstring("""
S    -> NP VP              [1.0]
VP   -> TV NP              [0.4]
VP   -> IV                 [0.3]
VP   -> DatV NP NP         [0.3]
TV   -> 'saw'              [1.0]
IV   -> 'ate'              [1.0]
DatV -> 'gave'             [1.0]
NP   -> 'telescopes'       [0.8]
NP   -> 'Jack'             [0.2]
""")

In [ ]:
viterbi_parser = nltk.ViterbiParser(grammar)
for i, tree in enumerate(viterbi_parser.parse(['Jack', 'saw', 'telescopes'])):

    print(tree)
tree.draw()    

--------------- >第 1 个结构< ---------------
(S (NP Jack) (VP (TV saw) (NP telescopes))) (p=0.064)
